# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Ämter der Bischöfe einlesen

Übertrage Daten aus Access. Die Daten werden aus Access via ODBC in eine Transfer-Datenbank geschrieben: `bischof.tbl_bischoefe_gatz` und `bischof.tbl_bischoefe_gatzaemter`. Alternativ werden die Daten in CSV-Dateien geschrieben und von dort aus hier eingelesen.

[Amt](#Amt)  
[Amtsperiode](#Amtsperiode)  
[Zeitspanne der Person](#Zeitspanne-der-Person)  
[Sortierung der Person nach Amt](#Sortierung-der-Person-nach-Amt)





Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path = "../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

*2022-11-03* Optimiere das Datenupdate. Test mit `wiag_in`  

In [10]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

*2022-10-15* mit Daten vom Juli; Update wegen neuer Ämtertabelle
*2022-12-03* neue Daten!

In [11]:
data_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bischoefe-2022-11-03"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bischoefe-2022-11-03"

In [12]:
item_type_id = 4

4

In [13]:
ENV["COLUMNS"] = 120

120

*2022-01-13* Für die offizielle Liste der Ämter gibt es inzwischen eine Liste: `domherr.tbl_aemterliste_normiert`. Die Ämter der Bischöfe sind neu zuzuordnen.

## Amtsperiode

In [10]:
# Quelle Datenbank
sql = "SELECT * FROM bischof.tbl_bischoefe_gatzaemter"
df_pr = Wds.sql_df(sql);

In [14]:
using CSV, DataFrames

In [15]:
# Quelle CSV
filename = joinpath(data_path, "tbl_bischoefe_gatzaemter.csv");
df_pr = CSV.read(filename, DataFrame, stringtype = String);

In [16]:
size(df_pr)

(7339, 16)

In [17]:
names(df_pr)

16-element Vector{String}:
 "Bistum"
 "Amtsart"
 "Amtsbeginn"
 "Amtsende"
 "ID_Amt"
 "ID_Bischof"
 "Institution"
 "Amt_weltlich"
 "Herrschaftsgebiet"
 "Ort"
 "Amt_Kommentar"
 "Sort"
 "Titularbistum"
 "Profession"
 "id_gsn"
 "Amt_normiert"

nicht befüllt:
- Herrschafsgebiet
- Institution
- Amt_weltlich
- Profession
- Ort

### Bischöfe zuordnen

In [18]:
transform!(df_pr, :ID_Bischof => ByRow(string) => :id_in_source);

In [19]:
bishop_item_type = 4;
sql = "SELECT id AS person_id, id_in_source FROM item WHERE item_type_id = $(bishop_item_type)";
df_idx = Wds.sql_df(sql);

In [20]:
df_prp = leftjoin(df_pr, df_idx, on = :id_in_source);

In [21]:
size(df_prp)

(7339, 18)

Amtsangaben ohne Bezug zu einem Bischof

In [22]:
sort_ID_Bischof(df) = sort(df, :ID_Bischof)
filter(:person_id => ismissing, df_prp) |> sort_ID_Bischof

Row,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof,Institution,Amt_weltlich,Herrschaftsgebiet,Ort,Amt_Kommentar,Sort,Titularbistum,Profession,id_gsn,Amt_normiert,id_in_source,person_id
,String?,String,String?,String?,Int64,Int64,Missing,Missing,Missing,Missing,String?,Int64,String?,Missing,Int64?,String?,String,Int32?
1,Brixen,Weihbischof,1379,1387,22075,21066,missing,missing,missing,missing,missing,11400,missing,missing,missing,Weihbischof,21066,missing
2,Chur,Weihbischof,um 1289,1294,22087,21078,missing,missing,missing,missing,missing,10100,missing,missing,missing,Weihbischof,21078,missing
3,Eichstätt,Weihbischof,1340,missing,22102,21093,missing,missing,missing,missing,missing,10000,missing,missing,missing,Weihbischof,21093,missing
4,Halberstadt,Weihbischof,1420,1436,22144,21135,missing,missing,missing,missing,missing,12000,missing,missing,missing,Weihbischof,21135,missing
5,Köln,Weihbischof,1228,missing,22173,21164,missing,missing,missing,missing,missing,10400,missing,missing,missing,Weihbischof,21164,missing
6,Köln,Weihbischof,1352,1364,22184,21175,missing,missing,missing,missing,missing,11800,missing,missing,missing,Weihbischof,21175,missing
7,Konstanz,Weihbischof,1289,1294 (1299 ?),22198,21189,missing,missing,missing,missing,missing,10900,missing,missing,missing,Weihbischof,21189,missing
8,Leitomischl,Weihbischof,1361,missing,22238,21229,missing,missing,missing,missing,missing,10000,missing,missing,missing,Weihbischof,21229,missing
9,Lüttich,Weihbischof,1233,missing,22243,21234,missing,missing,missing,missing,missing,10700,missing,missing,missing,Weihbischof,21234,missing


19 Amtsperioden können keinem Bischof zugeordnet werden?!

In [23]:
df_prp = dropmissing(df_prp, :person_id);

In [24]:
size(df_prp)

(7320, 18)

Lösche Einträge ohne Amtsbezeichnung

In [25]:
df_prp = dropmissing(df_prp, :Amtsart);

In [26]:
size(df_prp)

(7320, 18)

### Rollen/Ämter zuordnen

In [27]:
sql = "SELECT name, id FROM role"
df_role = Wds.sql_df(sql);

In [28]:
lookup_role = Dict(df_role.name .=> df_role.id);

In [29]:
get_role_id(name) = get(lookup_role, name, missing)

get_role_id (generic function with 1 method)

In [30]:
get_role_id("Vikar")

489680

Die Einträge in Tabelle `role` beginnen alle mit einem Großbuchstaben

In [31]:
starts_lowercase(s) = islowercase(s[1]);
df_prp_sl = filter(:Amtsart => starts_lowercase, df_prp);

In [32]:
select(df_prp_sl, [:person_id, :id_in_source, :Amtsart, :Bistum])

Row,person_id,id_in_source,Amtsart,Bistum
,Int32,String,String,String?
1,560621,10429,nominierter Bischof,Lüttich
2,560900,4254,designierter Bischof,Konstanz
3,563684,21771,evangelischer Administrator,Bremen-Hamburg
4,563685,21772,evangelischer Administrator,Bremen-Hamburg
5,563686,21773,evangelischer Administrator,Bremen-Hamburg


In [33]:
transform!(df_prp, :Amtsart => ByRow(uppercasefirst) => :Amtsart);

SQL berücksichtigt keine Unterschiede in Groß- und Kleinschreibung

In [34]:
df_info = subset(df_prp, :ID_Bischof => ByRow(isequal(10127)))

Row,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof,Institution,Amt_weltlich,Herrschaftsgebiet,Ort,Amt_Kommentar,Sort,Titularbistum,Profession,id_gsn,Amt_normiert,id_in_source,person_id
,String?,String,String?,String?,Int64,Int64,Missing,Missing,Missing,Missing,String?,Int64,String?,Missing,Int64?,String?,String,Int32
1,Münster,Fürstbischof,1719,1761,5609,10127,missing,missing,missing,missing,missing,6200,missing,missing,missing,Fürstbischof,10127,561814
2,Regensburg,Fürstbischof,1717,1719,4222,10127,missing,missing,missing,missing,missing,7000,missing,missing,missing,Fürstbischof,10127,561814
3,Paderborn,Fürstbischof,1719,1761,4224,10127,missing,missing,missing,missing,missing,5300,missing,missing,missing,Fürstbischof,10127,561814
4,Köln,Koadjutor des Erzbischofs,1722,1723,4225,10127,missing,missing,missing,missing,missing,8250,missing,missing,missing,missing,10127,561814
5,Köln,Kurfürst-Erzbischof,1723,1761,4226,10127,missing,missing,missing,missing,missing,8300,missing,missing,missing,missing,10127,561814
6,Hildesheim,Fürstbischof,1724,1761,4227,10127,missing,missing,missing,missing,missing,6800,missing,missing,missing,Fürstbischof,10127,561814
7,Osnabrück,Fürstbischof,1728,1761,4228,10127,missing,missing,missing,missing,missing,6100,missing,missing,missing,Fürstbischof,10127,561814
8,missing,Fürstpropst,1723,1761,21892,10127,missing,missing,missing,missing,1716-1723 Koadjutor des Fürstpropstes von Berchtesgaden,0,missing,missing,30121,missing,10127,561814
9,Regensburg,Koadjutor,1715,1716,23084,10127,missing,missing,missing,missing,missing,6970,missing,missing,missing,Koadjutor,10127,561814


Für Bischof 3807 sind Amtsart und Bistum vertauscht. *2022-05-19* in der Quelle korrigiert.

In [34]:
subset(df_pr, :ID_Bischof => ByRow(isequal(10127)))

,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof,Institution,Amt_weltlich,Herrschaftsgebiet
,String?,String,String?,String?,Int64,Int64,Missing,Missing,Missing
1,Münster,Fürstbischof,1719,1761,5609,10127,missing,missing,missing
2,Regensburg,Fürstbischof,1717,1719,4222,10127,missing,missing,missing
3,Paderborn,Fürstbischof,1719,1761,4224,10127,missing,missing,missing
4,Köln,Koadjutor des Erzbischofs,1722,1723,4225,10127,missing,missing,missing
5,Köln,Kurfürst-Erzbischof,1723,1761,4226,10127,missing,missing,missing
6,Hildesheim,Fürstbischof,1724,1761,4227,10127,missing,missing,missing
7,Osnabrück,Fürstbischof,1728,1761,4228,10127,missing,missing,missing
8,missing,Fürstpropst,1723,1761,21892,10127,missing,missing,missing
9,Regensburg,Koadjutor,1715,1716,23084,10127,missing,missing,missing


Ämter der Ämterliste zuordnen

In [35]:
transform!(df_prp, :Amtsart => ByRow(get_role_id) => :role_id);

In [36]:
df_role_mg = filter(:role_id => ismissing, df_prp);

In [37]:
length(unique(df_role_mg.Amtsart))

97

In [38]:
unique(df_role_mg[!, :Amtsart])[1:7]

7-element Vector{String}:
 "Fürstpropst"
 "Direktor des Konsistoriums"
 "Kurfürst-Erzbischof"
 "Apostolischer Administrator"
 "Fürstbischof"
 "Koadjutor des Bischofs"
 "Koadjutor des Kurfürst-Erzbischofs"

In diesen Fällen wird die Amtsart lediglich eingetragen in `person_role.role_name`

### Bistümer zuordnen

In [39]:
sql = "SELECT id AS diocese_id, name AS diocese_name FROM diocese";
df_dioc = Wds.sql_df(sql);

In [40]:
df_prp = leftjoin(df_prp, df_dioc, on = :Bistum => :diocese_name, matchmissing = :notequal);

In [41]:
df_dioc_mg = filter(:diocese_id => ismissing, df_prp);

In [42]:
size(df_dioc_mg)

(447, 20)

Numerische Daten eintragen

In [43]:
num_date_begin(s_date) = ismissing(s_date) ? s_date : Wds.parsemaybe(String(s_date), :lower)

num_date_begin (generic function with 1 method)

In [44]:
num_date_end(s_date) = ismissing(s_date) ? s_date : Wds.parsemaybe(String(s_date), :upper)

num_date_end (generic function with 1 method)

In [45]:
df_prp[201:205, :]

Row,Bistum,Amtsart,Amtsbeginn,Amtsende,ID_Amt,ID_Bischof,Institution,Amt_weltlich,Herrschaftsgebiet,Ort,Amt_Kommentar,Sort,Titularbistum,Profession,id_gsn,Amt_normiert,id_in_source,person_id,role_id,diocese_id
,String?,String,String?,String?,Int64,Int64,Missing,Missing,Missing,Missing,String?,Int64,String?,Missing,Int64?,String?,String,Int32,Int32?,Int32?
1,Trier,Kurfürst-Erzbischof,1756,1768,5674,10860,missing,missing,missing,missing,missing,9000,missing,missing,missing,missing,10860,562705,missing,30
2,Worms,Fürstbischof,1763,1768,5675,10860,missing,missing,missing,missing,missing,8100,missing,missing,missing,Fürstbischof,10860,562705,missing,32
3,Wien,Administrator,1476,1482,5676,5302,missing,missing,missing,missing,missing,300,missing,missing,missing,Administrator,5302,558794,489577,54
4,Salzburg,Administrator,1481,1487,5677,5302,missing,missing,missing,missing,missing,5330,missing,missing,missing,Administrator,5302,558794,489577,35
5,Magdeburg,Erzbischof,1125/1126,missing,5678,5080,missing,missing,missing,missing,missing,1400,missing,missing,missing,Erzbischof,5080,559502,489594,15


In [46]:
transform!(df_prp, :Amtsbeginn => ByRow(num_date_begin) => :num_date_begin);

┌ Warning: Could not parse 
│   s = fränkische Zeit ?
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1125


In [47]:
transform!(df_prp, :Amtsende => ByRow(num_date_end) => :num_date_end);

In [48]:
df_prp[561:565, [:ID_Bischof, :Amtsart, :Amtsbeginn, :num_date_begin, :Amtsende, :num_date_end]]

Row,ID_Bischof,Amtsart,Amtsbeginn,num_date_begin,Amtsende,num_date_end
,Int64,String,String?,Int64?,String?,Int64?
1,10475,Kurfürst-Erzbischof,1784,1784,1801,1801
2,10476,Fürstbischof,1650,1650,1688,1688
3,3302,Elekt,1387,1387,1389,1389
4,3307,Weihbischof,1279,1279,1280,1280
5,3307,Weihbischof,1279,1279,1285,1285


Schlüssel für eine zeitliche Sortierung eintragen

In [49]:
transform!(df_prp, :Amtsbeginn => ByRow(Wds.parse_year_sort) => :date_sort_key);

┌ Warning: could not parse 
│   s = (1168) 1180
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (1014 ?) 1015
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (952)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (972)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (984)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (991)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (996)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 

In [50]:
transform!(df_prp, :Amtsende => ByRow(Wds.parse_year_sort) => :date_sort_key_replace);

┌ Warning: could not parse 
│   s = (1014 ?) 1015
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (952)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (972)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (984)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (991)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (996)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   s = (1072)
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1539
┌ Warning: could not parse 
│   

In [51]:
replace_if_missing(a, b) = ismissing(a) || a == 9000900 ? b : a

replace_if_missing (generic function with 1 method)

In [52]:
transform!(df_prp, [:date_sort_key, :date_sort_key_replace] => ByRow(replace_if_missing) => :date_sort_key);

*2022-09-26* Bei manchen Bischöfen fehlt `date_sort_key` obwohl es eine Angabe zum Amtsende gibt.  
begin debug 
Korrektur: prüfe nicht nur auf `missing` sondern auch auf 9000900

In [53]:
filter_ids(x) = x in (319350, 319850, 320563, 322960)

filter_ids (generic function with 1 method)

In [54]:
df_prp_info = filter(:person_id => filter_ids, df_prp);

In [55]:
df_prp_info[:, [:person_id, :Bistum, :Amtsart, :Amtsbeginn, :Amtsende, :num_date_begin, :num_date_end, :date_sort_key, :date_sort_key_replace]]

0×9 DataFrame

end debug

In [56]:
size(df_prp, 1), count(ismissing, df_prp.date_sort_key)

(7320, 0)

Bestehende Einträge löschen

In [57]:
table_name = "person_role";
item_type_id = 4;
sql = "DELETE FROM $(table_name) WHERE person_id IN (SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM person_role WHERE person_id IN (SELECT id FROM item WHERE item_type_id = 4)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Daten schreiben

Schreibe für alle Bistümer, auch für die identifizierten, den Namen des Bistums in die Tabelle.  
Schreibe für alle Ämter, auch für die identifizierten, den Namen des Amts in die Tabelle.

In [59]:
columns = [
    :Bistum => :diocese_name,
    :Amtsbeginn => :date_begin,
    :Amtsende => :date_end,
    :Amt_Kommentar => :note,
    :Sort => :display_order,
    :Amtsart => :role_name,
    :person_id => :person_id,
    :diocese_id => :diocese_id,
    :role_id => :role_id,
    :num_date_begin => :num_date_begin,
    :num_date_end => :num_date_end,
    :date_sort_key => :date_sort_key,
];

In [60]:
table_dst = "person_role"
Wds.filltable!(table_dst, select(df_prp, columns))

┌ Info: Rows inserted: 7320
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


7320

Titularbistum eintragen

Identifikation über `ID_Bischof`, `Amtsart`, `Bistum`

Es gibt nur einen Eintrag, der nicht leer ist?!
``` sql
SELECT ID_Bischof, Amtsart, Bistum, COUNT(*) AS n
FROM bischof.tbl_bischoefe_gatzaemter
WHERE Titularbistum IS NOT NULL AND Titularbistum <> ''
GROUP BY ID_Bischof, Amtsart, Bistum
ORDER BY n DESC;
```

Es gibt nur einen Eintrag für `ID_Bischof` = 21596. Dieser Bischof ist aber nicht in der Bischofsliste.

In [60]:
filter(:ID_Bischof => isequal(21596), select(df_prp, :ID_Bischof, :Bistum, :Amtsart, :Titularbistum))

,ID_Bischof,Bistum,Amtsart,Titularbistum
,Int64,String?,String,String?


## Zeitspanne der Person

Numerische Werte der Lebensdaten

In [61]:
sql = "SELECT id, date_birth, date_death, num_date_birth, num_date_death
FROM person WHERE id in (SELECT id FROM item WHERE item_type_id = $(item_type_id))"
df_date = Wds.sql_df(sql);

In [62]:
parse_date_birth(s) = Wds.parsemaybe(s, :lower)

parse_date_birth (generic function with 1 method)

In [63]:
transform!(df_date, :date_birth => ByRow(parse_date_birth) => :num_date_birth);

In [64]:
parse_date_death(s) = Wds.parsemaybe(s, :upper)

parse_date_death (generic function with 1 method)

In [65]:
transform!(df_date, :date_death => ByRow(parse_date_death) => :num_date_death);

In [66]:
df_date[231:240, :]

Row,id,date_birth,date_death,num_date_birth,num_date_death
,Int32,String?,String?,Int64?,Int64?
1,558747,missing,1404,missing,1404
2,558748,missing,spätestens 1398,missing,1398
3,558749,missing,1386,missing,1386
4,558750,1322/24,1395,1322,1395
5,558751,missing,frühestens 1400,missing,1450
6,558752,1335,1375,1335,1375
7,558753,missing,1361,missing,1361
8,558754,missing,1338,missing,1338
9,558755,missing,1308,missing,1308


Update der Datenbank: person.num_date_birth; person.num_date_death

In [67]:
for row in eachrow(df_date)
    id, num_date_birth, num_date_death = row[[:id, :num_date_birth, :num_date_death]]
    if !(ismissing(num_date_birth))
        sql = "UPDATE person SET num_date_birth = $(num_date_birth) WHERE id = $(id)"
        DBInterface.execute(Wds.dbwiag, sql)
    end
    if !(ismissing(num_date_death))
        sql = "UPDATE person SET num_date_death = $(num_date_death) WHERE id = $(id)"
        DBInterface.execute(Wds.dbwiag, sql)
    end
end    

Zeitspanne der Ämter

In [68]:
sql = "SELECT person_id, role_id, role_name, diocese_id, diocese_name, num_date_begin, num_date_end
FROM person_role pr
INNER JOIN person p ON p.id = pr.person_id
WHERE p.item_type_id = $(item_type_id)"
df_pr_date = Wds.sql_df(sql);

In [69]:
df_pr_begin_value = dropmissing(df_pr_date, :num_date_begin);
df_pr_begin = groupby(select(df_pr_begin_value, :person_id, :num_date_begin), :person_id);

In [70]:
df_pr_date_min = combine(df_pr_begin, :num_date_begin => minimum => :date_min);

In [71]:
size(df_pr_date_min)

(5025, 2)

In [72]:
names(df_pr_date_min)

2-element Vector{String}:
 "person_id"
 "date_min"

In [73]:
df_pr_end_value = dropmissing(df_pr_date, :num_date_end);
df_pr_end = groupby(select(df_pr_end_value, :person_id, :num_date_end), :person_id);

In [74]:
df_pr_date_max = combine(df_pr_end, :num_date_end => maximum => :date_max);

In [75]:
size(df_pr_date_max)

(4539, 2)

In [76]:
df_date = leftjoin(df_date, df_pr_date_min, on = :id => :person_id);

In [77]:
df_date = leftjoin(df_date, df_pr_date_max, on = :id => :person_id);

In [78]:
size(df_date)

(5171, 7)

In [79]:
names(df_date)

7-element Vector{String}:
 "id"
 "date_birth"
 "date_death"
 "num_date_birth"
 "num_date_death"
 "date_min"
 "date_max"

In [80]:
df_date[231:235, :]

Row,id,date_birth,date_death,num_date_birth,num_date_death,date_min,date_max
,Int32,String?,String?,Int64?,Int64?,Int32?,Int32?
1,558827,missing,1359,missing,1359,1354,1359
2,558828,missing,1331/32,missing,1331,1328,1331
3,558829,missing,vor 1332,missing,1332,1322,1330
4,558830,missing,spätestens 1322,missing,1322,1250,1321
5,558831,missing,1292,missing,1292,1263,1292


Geburts- und Sterbedatum  
Geburts- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [81]:
min_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : min(x, y));
max_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : max(x, y));

In [82]:
transform!(df_date, [:date_min, :num_date_birth] => ByRow(max_may_be) => :date_min); # sic!

In [83]:
transform!(df_date, [:date_max, :num_date_death] => ByRow(min_may_be) => :date_max); # sic!

Übernimm' den jeweils gültigen Wert für fehlende Daten

In [84]:
get_first_not_missing(x, y) = !ismissing(x) ? x : y;

In [85]:
transform!(df_date, [:date_min, :date_max] => ByRow(get_first_not_missing) => :date_min);

In [86]:
transform!(df_date, [:date_max, :date_min] => ByRow(get_first_not_missing) => :date_max);

Trage die geänderten Daten in die Tabelle `person` ein

In [87]:
for row in eachrow(df_date)
    id, date_min, date_max = row[[:id, :date_min, :date_max]]
    if !(ismissing(date_min))
        sql = "UPDATE person SET date_min = $(date_min) WHERE id = $(id)"
        DBInterface.execute(Wds.dbwiag, sql)
    end
    if !(ismissing(date_max))
        sql = "UPDATE person SET date_max = $(date_max) WHERE id = $(id)"
        DBInterface.execute(Wds.dbwiag, sql)
    end
end

## Sortierung der Person nach Amt
Die Tabelle `person_display_order` ist eine technische Hilfstabelle, um die Abfragen in Doctrine zu vereinfachen oder überhaupt erst zu ermöglichen. Jede Person erhält für jedes Bistum einen Sortierschlüssel und einen weiteren unabhängig vom Bistum.

*2022-01-14* Verzichte zunächst auf diese Tabelle und löse das Problem in der Web-Anwendung. So bleibt das Datenmodell schlanker.

In [45]:
using MySQL, DataFrames

Sortierung unabhängig von einer Diözese. Es werden auch Ämter berücksichtigt, die nicht mit einer Diözese verbunden sind.

In [48]:
sql = "SELECT person_id, 'any' as diocese, min(display_order) as display_order " *
"FROM person_role GROUP BY person_id";
df_do = db_exec(sql);

In [49]:
size(df_do)

(5159, 3)

Zwölf Bischöfe haben kein Amt!?

In [50]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 5159
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


5159

Sortierung nach Diözese

In [52]:
sql = "SELECT person_id, diocese_name as diocese, min(display_order) as display_order " *
"FROM person_role " *
"WHERE diocese_name IS NOT NULL " *
"GROUP BY person_id, diocese_name";
df_do = db_exec(sql);

In [53]:
size(df_do)

(6370, 3)

In [54]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 6370
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


6370

Bischöfe ohne Ämter

In [55]:
sql = "SELECT id AS person_id, 'any' as diocese, 950000 AS display_order " *
"FROM person WHERE id NOT IN (SELECT DISTINCT person_id FROM person_role)"
df_do_r = db_exec(sql);

In [56]:
size(df_do_r)

(12, 3)

In [57]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do_r)

┌ Info: Rows inserted: 12
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


12